In [1]:
import pandas as pd
import numpy as np


In [2]:
#Import file

file_path = "../Resources/purchase_data.csv"
#Read file 
purchase_data = pd.read_csv(file_path)
purchase_data.head()

 

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
#count number of player
total_players = len(purchase_data["SN"].unique())
 

summary_df = pd.DataFrame({"Total Unique Players": [total_players]})
summary_df                           

,Total Unique Players
0,576


In [4]:
purchase_data.dtypes

Purchase ID      int64
SN              object
Age              int64
Gender          object
Item ID          int64
Item Name       object
Price          float64
dtype: object

In [5]:
#Run basic calculations to obtain number of unique items, average price, etc.
unique_items = len(purchase_data["Item ID"].unique())
average_price = round(purchase_data["Price"].mean(),2)
average_age = round(purchase_data["Age"].mean(),0)
number_purchases = purchase_data["Item ID"].count()
revenue = purchase_data["Price"].sum()

#Create a summary data frame to hold the results
#Optional: give the displayed data cleaner formatting
purchasing_analysis = pd.DataFrame({"Number of Unique Items": [unique_items],
                             "Average Price": [average_price],
                              "Average Age": [average_age],
                              "Number Purchases": [number_purchases],
                              "Revenue": [revenue]})

 
purchasing_analysis["Average Price"]=purchasing_analysis["Average Price"].map("${:,.2f}".format)

purchasing_analysis["Revenue"]=purchasing_analysis["Revenue"].map("${:,.2f}".format)
 
#Display the summary data frame
purchasing_analysis

,Number of Unique Items,Average Price,Average Age,Number Purchases,Revenue
0,179,$3.05,23.0,780,"$2,379.77"


In [6]:
 
#count female players
female_players = purchase_data.loc[purchase_data["Gender"] =="Female"]
total_female_players = len(female_players["SN"].unique())

#percent female players
percent_female_players = round(total_female_players/total_players *100, 0)

#count other players
other_players = purchase_data.loc[purchase_data["Gender"] =="Other / Non-Disclosed"]
total_other_players = len(other_players["SN"].unique())

#percent other players
percent_other_players = round(total_other_players/total_players *100,0)

#count male players
male_players = purchase_data.loc[purchase_data["Gender"] == "Male", :]
total_male_players = len(male_players["SN"].unique())

#percent male players
percent_male_players = round(total_male_players/total_players *100,0)


#create DataFrame

gender_demographics = pd.DataFrame(np.array([["Female", percent_female_players, total_female_players], 
                                             ["Male", percent_male_players, total_male_players], 
                                             ["Other", percent_other_players, total_other_players]]),
                                             columns=['Gender', 'Percent', 'Count'])

gender_demographics = gender_demographics.set_index('Gender')
gender_demographics["Percent"]=gender_demographics["Percent"].astype(float).map("{:,.2f}%".format)

gender_demographics

,Percent,Count
Gender,,
Female,14.00%,81
Male,84.00%,484
Other,2.00%,11


In [7]:
#establish bins
bins = [0, 15, 25, 40, 50]

# Create labels for these bins
group_labels = ["0-15", "16-25", "26-40", "40-50"]

#cut data and put in bins

purchase_data["Age Group"] = pd.cut(purchase_data["Age"], bins, labels=group_labels)

#count players
count_players = pd.DataFrame({"Total Count":purchase_data.groupby(["Age Group"])["SN"].count()}).reset_index()

#percent_players
percentages=round(count_players["Total Count"]/count_players["Total Count"].sum()*100, 0 )
count_players["Percentages"]=percentages.map("{:,.2f}%".format)
count_players = count_players.set_index('Age Group')
count_players






,Total Count,Percentages
Age Group,,
0-15,86,11.00%
16-25,525,67.00%
26-40,162,21.00%
40-50,7,1.00%


In [8]:

#establish bins
bins = [0, 15, 25, 40, 50]

# Create labels for these bins
group_labels = ["0-15", "16-25", "26-40", "40-50"]

#cut data and put in bins

purchase_data["Age Group"] = pd.cut(purchase_data["Age"], bins, labels=group_labels)

#purchase Count
purchase_count = pd.DataFrame({"Purchase Count": purchase_data.groupby(["Age Group"])["Purchase ID"].count()}).reset_index()

#purchase price
purchase_price = pd.DataFrame({"Purchase Price": purchase_data.groupby(["Age Group"])["Price"].mean()}).reset_index().round(2)

#purchase Value
purchase_value = pd.DataFrame({"Purchase Value": purchase_data.groupby(["Age Group"])["Price"].sum()})

#purchase_total
average_purchase_price = pd.DataFrame({"Total Price": purchase_data.groupby(["Age Group", "SN"])["Price"].sum()}).reset_index().round(2)
purchase_total = pd.DataFrame({"Average Purchase Price per Person":average_purchase_price.groupby(["Age Group"])["Total Price"].mean()}).reset_index().round(2)
age_purchase_analysis = purchase_count.merge(purchase_price, on="Age Group", how="left")

age_purchase_analysis = age_purchase_analysis.merge(purchase_total, on="Age Group", how="left")
age_purchase_analysis = age_purchase_analysis.merge(purchase_value, on="Age Group", how="left")
age_purchase_analysis = age_purchase_analysis.set_index('Age Group')
age_purchase_analysis

,Purchase Count,Purchase Price,Average Purchase Price per Person,Purchase Value
Age Group,,,,
0-15,86,3.09,4.09,265.56
16-25,525,3.05,4.20,1603.20
26-40,162,3.02,4.01,489.48
40-50,7,3.08,3.08,21.53


In [9]:
purchase_data.head()

#purchase count

purchase_count = pd.DataFrame({"Purchase Count": purchase_data.groupby(["SN"])["Purchase ID"].count()}).reset_index()

#average purchase price
average_purchase_price = pd.DataFrame({"Average Purchase Price": purchase_data.groupby(["SN"])["Price"].mean().map("${:,.2f}".format)}).reset_index().round(2)


#total purchase value
average_purchase_price_sum = pd.DataFrame({"Total Price": purchase_data.groupby(["SN"])["Price"].sum()}).reset_index()
total_purchase_value = pd.DataFrame({"Total Purchase Value":average_purchase_price_sum.groupby(["SN"])["Total Price"].mean().map("${:,.2f}".format)}).reset_index()


big_spenders = purchase_count.merge(average_purchase_price, on="SN", how="left")
big_spenders = big_spenders.merge(total_purchase_value, on="SN", how="left")
big_spenders = big_spenders.set_index('SN')
big_spenders = big_spenders.sort_values(["Total Purchase Value"], ascending=False)
big_spenders.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Haillyrgue51,3,$3.17,$9.50
Phistym51,2,$4.75,$9.50
Lamil79,2,$4.64,$9.29
Aina42,3,$3.07,$9.22
Saesrideu94,2,$4.59,$9.18


In [10]:

total_purchase_value = pd.DataFrame({"Total Purchase Value":purchase_data.groupby(["Item Name"])["Price"].sum().map("${:,.2f}".format)}).reset_index()

purchase_count = pd.DataFrame({"Purchase Count":purchase_data.groupby(["Item Name"])["Price"].count()}).reset_index()

item_price = pd.DataFrame({"Item Price":purchase_data.groupby(["Item Name"])["Price"].mean().map("${:,.2f}".format)}).reset_index()

most_popular = total_purchase_value.merge(purchase_count, on="Item Name", how="left")

most_popular = most_popular.merge(item_price, on="Item Name", how="left")
most_popular = most_popular.set_index('Item Name')

most_popular = most_popular.sort_values(["Purchase Count"], ascending=False)
most_popular.head()

,Total Purchase Value,Purchase Count,Item Price
Item Name,,,
Final Critic,$59.99,13,$4.61
"Oathbreaker, Last Hope of the Breaking Storm",$50.76,12,$4.23
Persuasion,$28.99,9,$3.22
Nirvana,$44.10,9,$4.90
"Extraction, Quickblade Of Trembling Hands",$31.77,9,$3.53


In [11]:
most_popular = most_popular.sort_values(["Total Purchase Value"], ascending=False)
most_popular

,Total Purchase Value,Purchase Count,Item Price
Item Name,,,
Stormfury Mace,$9.98,2,$4.99
"Chaos, Ender of the End",$9.90,5,$1.98
Stormfury Longsword,$9.86,2,$4.93
"The Void, Vengeance of Dark Magic",$9.48,4,$2.37
Frenzied Scimitar,$9.36,6,$1.56
...,...,...,...
Crying Steel Sickle,$10.23,3,$3.41
Severance,$10.20,3,$3.40
Suspension,$10.08,7,$1.44
